In [1]:
from lxml import etree
import pandas as pd
import os
import re
from itertools import zip_longest
import csv
from collections import defaultdict
import unicodedata
import thesis_utils
from collections import Counter, defaultdict
import difflib

/home/elisabetta/anaconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
comet_ml is installed but `COMET_API_KEY` is not set.


In [2]:
multiged_merlin_train=  thesis_utils.read_tsv_file_and_find_sentences_without_headers('./MULTI-GED2023 DATA/it_merlin_train.tsv')
multiged_train_info= thesis_utils.get_list_ids_tokens_gold(multiged_merlin_train)
print(multiged_train_info[0])

[('T0', 'Ci', 'c'), ('T1', 'sono', 'c'), ('T2', 'stata', 'c'), ('T3', 'con', 'c'), ('T4', 'il', 'c'), ('T5', 'mio', 'c'), ('T6', 'fidanzato', 'c'), ('T7', 'per', 'c'), ('T8', 'due', 'c'), ('T9', 'settimane', 'c'), ('T10', '.', 'c')]


In [3]:
multiged_merlin_dev =  thesis_utils.read_tsv_file_and_find_sentences_without_headers('./MULTI-GED2023 DATA/it_merlin_dev.tsv')
multiged_dev_info= thesis_utils.get_list_ids_tokens_gold(multiged_merlin_dev)
print(multiged_dev_info[0])
print(len(multiged_dev_info))

[('T0', 'Residence', 'c'), ('T1', 'al', 'c'), ('T2', 'Lago', 'c'), ('T3', 'Maria', 'c'), ('T4', 'Rossi', 'c'), ('T5', 'Strada', 'c'), ('T6', 'Maggiore', 'c'), ('T7', '2', 'c'), ('T8', '/', 'c'), ('T9', 'D', 'c'), ('T10', 'Città', 'c'), ('T11', 'X', 'c'), ('T12', 'Ungheria', 'c'), ('T13', '13', 'c'), ('T14', '/', 'c'), ('T15', '08', 'c'), ('T16', '/', 'c'), ('T17', '2011', 'c'), ('T18', 'Egregio', 'c'), ('T19', 'Signore', 'i'), ('T20', '/', 'c'), ('T21', 'a', 'c'), ('T22', ',', 'c'), ('T23', 'La', 'i'), ('T24', 'scrivo', 'c'), ('T25', 'al', 'i'), ('T26', 'riferimento', 'c'), ('T27', 'dei', 'i'), ('T28', 'servizi', 'c'), ('T29', 'del', 'c'), ('T30', 'Suo', 'c'), ('T31', 'residence', 'c'), ('T32', '.', 'c')]
758


In [4]:
# multiged_merlin_test =  thesis_utils.read_tsv_file_and_find_sentences('./MULTI-GED2023 DATA/it_merlin_test_unlabelled.tsv')
# multiged_merlin_test_fixed = [[token[0] for token in sentence] for sentence in multiged_merlin_test]

In [5]:
all_combined_sentences_train = []
for sentence in multiged_train_info:
    cleaned_sentence = ''.join(token[1].replace('\\\\\\', '') for token in sentence)
    cleaned_labels = ''.join(token[-1].replace('\\\\\\', '') for token in sentence)# Clean and join tokens
    all_combined_sentences_train.append((cleaned_sentence, cleaned_labels))

In [6]:
print(all_combined_sentences_train[:10])

[('Cisonostataconilmiofidanzatoperduesettimane.', 'ccccccccccc'), ('Siamoappassianatidellosport.', 'ciccc'), ('Purtroppononabbiamopotutogiocaretennisperchèicampidatenniseranoinriparazione.', 'ccccciicccccccc'), ('Questoeraunproblemagranderiguardandoallacosacheilmiofidanzatounprofessionistadeltennis.', 'ccccciccccccicccc'), ('Quantoamelapiúgrandeproblemaeracheillagosiastatalontanissimo.', 'ccciicccccciicc'), ('Mentrel\'annunciodiceva"Illagosiraggiungecomodamenteconimezzipubblici".', 'iccccccccccccccc'), ("Unaaltramancezzaeral'équipedianimazione.", 'ccicccccc'), ("Melapiacemoltomac'eraunasolaseraquandoc'èraanimazione.", 'iiiccciicciiiic'), ('NonsonosoddisfattaneancheilmiofidanzatoècontentoconiSuoiservizi.', 'ccciccccciiccc'), ("LepreghiamocompensazioneesepotessidarLeunconsigliocambial'annuncio.", 'icicciccciccc')]


In [7]:
all_combined_sentences_dev = []
for sentence in multiged_dev_info:
    cleaned_sentence = ''.join(token[1].replace('\\\\\\', '') for token in sentence)
    cleaned_labels = ''.join(token[-1].replace('\\\\\\', '') for token in sentence)# Clean and join tokens
# Clean and join tokens
    all_combined_sentences_dev.append((cleaned_sentence, cleaned_labels))

In [8]:
print(all_combined_sentences_dev[0])

('ResidencealLagoMariaRossiStradaMaggiore2/DCittàXUngheria13/08/2011EgregioSignore/a,LascrivoalriferimentodeiservizidelSuoresidence.', 'ccccccccccccccccccciccciciciccccc')


In [9]:
# all_combined_sentences_test = []
# for sentence in multiged_merlin_test_fixed:
#     cleaned_sentence = ''.join(token.replace('\\\\\\', '') for token in sentence)  # Clean and join tokens
#     all_combined_sentences_test.append(cleaned_sentence)

In [10]:
def extract_sentences_and_tokens_with_annotations(directory_merlin):
    """
    Extracts tokens from TH1 and TH2 tiers and their diffs and markable_scheme annotations.
    Returns a list of dictionaries: [{filename: [(tid, token_th1, th1diff, token_th2, th2diff, markable_scheme), ...]}, ...]
    """
    all_files_data = []
    file_sentences = {}

    for filename in os.listdir(directory_merlin):
        if filename.endswith(".exb"):
            filepath = os.path.join(directory_merlin, filename)
            with open(filepath, "r", encoding="utf-8") as file:
                content = file.read()
                content = re.sub(r'&(?!\w+;|#\d+;)', '&amp;', content)

                try:
                    root = etree.fromstring(content.encode("utf-8"))

                    th1_tokens = {}
                    for tier in root.findall(".//tier[@category='TH1']"):
                        for event in tier.findall(".//event"):
                            tid = event.get("start")
                            token = event.text.strip() if event.text else ''
                            if tid and token:
                                th1_tokens[tid] = token

                    th2_tokens = {}
                    for tier in root.findall(".//tier[@category='TH2']"):
                        for event in tier.findall(".//event"):
                            tid = event.get("start")
                            token = event.text.strip() if event.text else ''
                            if tid and token:
                                th2_tokens[tid] = token

                    th1_diffs = {}
                    for tier in root.findall(".//tier[@category='TH1Diff']"):
                        for event in tier.findall(".//event"):
                            tid = event.get("start")
                            diff = event.text.strip() if event.text else ''
                            if tid:
                                th1_diffs[tid] = diff

                    th2_diffs = {}
                    for tier in root.findall(".//tier[@category='TH2Diff']"):
                        for event in tier.findall(".//event"):
                            tid = event.get("start")
                            diff = event.text.strip() if event.text else ''
                            if tid:
                                th2_diffs[tid] = diff

                    markable_scheme = {}
                    for tier in root.findall(".//tier[@category='markable_scheme']"):
                        for event in tier.findall(".//event"):
                            start_tid = event.get("start")
                            end_tid = event.get("end")
                            label = event.text.strip() if event.text else ''
                    
                            if start_tid and end_tid and label:
                                try:
                                    start_idx = int(start_tid[1:])
                                    end_idx = int(end_tid[1:])
                                    for i in range(start_idx, end_idx):
                                        tid = f"T{i}"
                                        markable_scheme[tid] = label
                                except ValueError:
                                    continue

                    all_tids = set(th1_tokens) | set(th2_tokens) | set(th1_diffs) | set(th2_diffs) | set(markable_scheme)

                    token_data = []
                    for tid in sorted(all_tids, key=lambda x: int(x[1:])):
                        token_data.append((
                            tid,
                            th1_tokens.get(tid, ''),
                            th1_diffs.get(tid, ''),
                            th2_tokens.get(tid, ''),
                            th2_diffs.get(tid, ''),
                            markable_scheme.get(tid, '')
                        ))

                    all_files_data.append({filename: token_data})

                    token_dict = {}
                    for tier in root.findall(".//tier[@category='learner']"):
                        for event in tier.findall(".//event"):
                            time_id = event.get("start")
                            token = event.text.strip() if event.text else ''
                            if time_id and token:
                                token_dict[time_id] = token

                    sentences = []
                    for tier in root.findall(".//tier[@category='sentence']"):
                        for event in tier.findall(".//event"):
                            start_time = event.get("start")
                            end_time = event.get("end")
                            
                            if not start_time or not end_time:
                                continue
                            
                            start_num = int(start_time[1:])
                            end_num = int(end_time[1:])

                            matching_tokens = [
                                (int(tid[1:]), tid) for tid in token_dict
                                if start_num <= int(tid[1:]) < end_num
                            ]

                            matching_tokens.sort()
                            sentence_tokens = [(tid, token_dict[tid]) for (_, tid) in matching_tokens]

                            if sentence_tokens:
                                sentences.append(sentence_tokens)

                    file_sentences[filename] = sentences

                except etree.XMLSyntaxError as e:
                    print(f"Error parsing {filename}: {e}")

    return all_files_data,file_sentences


In [11]:
th1_th2,sentences_merlin = extract_sentences_and_tokens_with_annotations('./merlin-exmaralda-v1.1/italian')

In [12]:
for file, sentences in sentences_merlin.items():
    print(file,sentences)

1385_0000053.exb [[('T0', 'Ciao'), ('T1', 'amico'), ('T3', 'grazie'), ('T4', 'per'), ('T5', 'la'), ('T6', 'tua'), ('T7', 'email'), ('T8', '.')], [('T9', 'Vorrei'), ('T10', 'aiutarti'), ('T11', ','), ('T12', 'ma'), ('T14', 'bisogno'), ('T15', 'di'), ('T16', 'qualche'), ('T17', 'informazione'), ('T18', 'in'), ('T19', 'più'), ('T21', 'Puoi'), ('T22', 'dirmi'), ('T23', 'a'), ('T24', 'questa'), ('T25', 'dita'), ('T26', 'hai'), ('T27', 'lavorato'), ('T28', 'in'), ('T29', 'tempo'), ('T30', 'scorso'), ('T31', 'e'), ('T32', 'che'), ('T33', 'chosa'), ('T34', 'hai'), ('T35', 'fatto'), ('T36', '?')], [('T37', 'Anche'), ('T38', 'vorrei'), ('T39', 'sapere'), ('T41', 'che'), ('T42', 'tipo'), ('T43', 'di'), ('T44', 'lavoro'), ('T45', 'ti'), ('T46', 'piace'), ('T47', ','), ('T48', 'preferici'), ('T50', 'lavoro'), ('T51', 'tecnico'), ('T52', 'o'), ('T54', 'lavoro'), ('T55', 'i'), ('T56', 'ufficio'), ('T57', '?')], [('T58', 'Sai'), ('T60', 'lingue'), ('T61', 'diverse'), ('T62', '?')], [('T63', 'Potresti'

In [13]:
def is_problematic_token(token):
    """Identify tokens to remove (unreadable, hyphens, etc.)"""
    return (re.match(r'^-.+-$', token) or 
            token == '-unreadable-' or
            token.strip() == '')

def process_sentences(sentences_merlin):
    merlin_combined = []
    
    for file_name, sentences in sentences_merlin.items():
        clean_entries = []
        
        for sentence in sentences:
            filtered = [(idx, token) for idx, token in sentence if not is_problematic_token(token)]
            
            if not filtered:
                continue
                
            clean_sentence = ''.join([token for _, token in filtered])
            clean_indices = ''.join([idx * len(token) for idx, token in filtered])
            
            clean_entries.append((clean_sentence, clean_indices))
        
        for sent, idx in clean_entries:
            merlin_combined.append((file_name, sent, idx))
        
        for i in range(1, len(clean_entries)):
            combined_sent = clean_entries[i-1][0] + clean_entries[i][0]
            combined_idx = clean_entries[i-1][1] + clean_entries[i][1]
            merlin_combined.append((file_name, combined_sent, combined_idx))
    
    return merlin_combined

merlin_combined = process_sentences(sentences_merlin)
print(f"Total sentences processed: {len(merlin_combined)}")

Total sentences processed: 16443


In [14]:
print(merlin_combined[0])
print(merlin_combined[1])
print(merlin_combined[2])

('1385_0000053.exb', 'Ciaoamicograzieperlatuaemail.', 'T0T0T0T0T1T1T1T1T1T3T3T3T3T3T3T4T4T4T5T5T6T6T6T7T7T7T7T7T8')
('1385_0000053.exb', 'Vorreiaiutarti,mabisognodiqualcheinformazioneinpiùPuoidirmiaquestaditahailavoratointemposcorsoechechosahaifatto?', 'T9T9T9T9T9T9T10T10T10T10T10T10T10T10T11T12T12T14T14T14T14T14T14T14T15T15T16T16T16T16T16T16T16T17T17T17T17T17T17T17T17T17T17T17T17T18T18T19T19T19T21T21T21T21T22T22T22T22T22T23T24T24T24T24T24T24T25T25T25T25T26T26T26T27T27T27T27T27T27T27T27T28T28T29T29T29T29T29T30T30T30T30T30T30T31T32T32T32T33T33T33T33T33T34T34T34T35T35T35T35T35T36')
('1385_0000053.exb', 'Anchevorreisaperechetipodilavorotipiace,prefericilavorotecnicoolavoroiufficio?', 'T37T37T37T37T37T38T38T38T38T38T38T39T39T39T39T39T39T41T41T41T42T42T42T42T43T43T44T44T44T44T44T44T45T45T46T46T46T46T46T47T48T48T48T48T48T48T48T48T48T50T50T50T50T50T50T51T51T51T51T51T51T51T52T54T54T54T54T54T54T55T56T56T56T56T56T56T56T57')


In [15]:
train_counter = Counter(all_combined_sentences_train)
dev_counter = Counter(all_combined_sentences_dev)


train_available = train_counter.copy()
dev_available = dev_counter.copy()

merlin_sentences_by_file = defaultdict(list)
for file_name, sentence, indexes in merlin_combined:
    merlin_sentences_by_file[sentence].append((file_name, indexes))
    
matching_train = []
matching_dev = []
not_matching_any = []

for sentence, file_list in merlin_sentences_by_file.items():
    remaining = len(file_list)

    matching_keys = [(k, v) for k, v in train_available.items() if k[0] == sentence and v > 0]

    for (key_sentence, second_part), count in matching_keys:
        match_count = min(remaining, count)

        for _ in range(match_count):
            file_name, indexes = file_list.pop()
            matching_train.append((file_name, sentence, second_part, indexes))

        train_available[(key_sentence, second_part)] -= match_count
        remaining -= match_count

        if remaining == 0:
            break

    matching_keys_dev = [(k, v) for k, v in dev_available.items() if k[0] == sentence and v > 0]

    for (key_sentence, second_part), count in matching_keys_dev:
        match_count = min(remaining, count)

        for _ in range(match_count):
            file_name, indexes = file_list.pop()
            matching_dev.append((file_name, sentence, second_part, indexes))

        dev_available[(key_sentence, second_part)] -= match_count
        remaining -= match_count

        if remaining == 0:
            break

    for _ in range(remaining):
        file_name, indexes = file_list.pop()
        not_matching_any.append((file_name, sentence, indexes))

print(f"Number of matching sentences in train: {len(matching_train)}")
print(f"Number of matching sentences in dev: {len(matching_dev)}")
print(f"Number of sentences not matching any category: {len(not_matching_any)}")

Number of matching sentences in train: 5898
Number of matching sentences in dev: 700
Number of sentences not matching any category: 9845


In [16]:
sentences = [t[1] for t in matching_dev]
sentence_counts = Counter(sentences)

duplicates = {sent: count for sent, count in sentence_counts.items() if count > 1}

print("Duplicate sentences and their counts:")
for sent, count in duplicates.items():
    print(f"{repr(sent)} → {count} times")

print()

sentence_indices = defaultdict(list)

for idx, sentence in enumerate(all_combined_sentences_dev):
    sentence_indices[sentence].append(idx)

duplicates = {s: idxs for s, idxs in sentence_indices.items() if len(idxs) > 1}

print("Multiged duplicate sentences and their counts:")
for sentence, indices in duplicates.items():
    print(f"{repr(sentence)} → {len(indices)} times")

Duplicate sentences and their counts:
'Vorreiaiutarti,mahobisognodiqualcheinformazioneinpiù.' → 4 times
'CaroMichele,grazieperlatuaemail.' → 3 times
'Andiamoacenainunristorantetailandese.' → 2 times
'Comestai?' → 4 times
'Grazieperlatuaemail.' → 2 times
"Com'èlatuacittà?" → 2 times
'SonoaCittàXperfareuncorsodiitaliano.' → 2 times
'Chenepensi?' → 3 times
'Aspettolatuarisposta!' → 2 times
'CaroDaniele,comestai?' → 2 times

Multiged duplicate sentences and their counts:
('!', 'c') → 3 times
('Aspettolatuarisposta!', 'ccccc') → 2 times
('Chenepensi?', 'cccc') → 3 times
('Vorreiaiutarti,mahobisognodiqualcheinformazioneinpiù.', 'cccccccccccc') → 4 times
('Comestai?', 'ccc') → 2 times
('CaroDaniele,comestai?', 'cccccc') → 2 times
('Andiamoacenainunristorantetailandese.', 'cccccccc') → 2 times
('CaroMichele,grazieperlatuaemail.', 'ccccccccc') → 3 times
('Comestai?', 'icc') → 2 times
('SonoaCittàXperfareuncorsodiitaliano.', 'ccccccccccc') → 2 times
("Com'èlatuacittà?", 'cccccc') → 2 times


In [17]:
print(matching_dev[1])

('1395_0001067.exb', 'Tuttidellamiafamigliastannobenissimo.', 'ciccccc', 'T24T24T24T24T24T25T25T25T25T25T26T26T26T27T27T27T27T27T27T27T27T28T28T28T28T28T28T29T29T29T29T29T29T29T29T29T30')


In [18]:
def multiged_unmatched(multiged_available):
    multiged_remaining_unmatched = []
    for sentence, count in multiged_available.items():
        if count > 0:
            multiged_remaining_unmatched.extend([sentence] * count)
    return multiged_remaining_unmatched
    
dev_remaining_unmatched = multiged_unmatched(dev_available)
train_remaining_unmatched = multiged_unmatched(train_available)

print(f"\nNumber of dev sentences still to match: {len(dev_remaining_unmatched)}")


Number of dev sentences still to match: 58


In [19]:
def split_token_ids(index_string):
    return re.findall(r'T\d+', index_string)

def get_charwise_token_ids(sentence1, sentence2, indexes1):
    token_ids = split_token_ids(indexes1)
    if len(token_ids) != len(sentence1):
        raise ValueError(f"Mismatch between characters and token IDs: {len(sentence1)} chars vs {len(token_ids)} IDs")
    
    start = sentence1.find(sentence2)
    if start == -1:
        raise ValueError(f"Sentence2 not found in sentence1:\n→ sentence1: {sentence1}\n→ sentence2: {sentence2}")
    end = start + len(sentence2)
    
    return ''.join(token_ids[start:end])

def find_and_remove_overlapping_matches(not_matching_any, dev_remaining_unmatched, threshold=0.8):
    results = set()
    matched_not_any = set()
    matched_dev = set()

    for file_name, sentence1, indexes1 in not_matching_any:
        for sentence2, gold_label in dev_remaining_unmatched:
            similarity = difflib.SequenceMatcher(None, sentence1, sentence2).ratio()
            if similarity >= threshold:
                if (sentence2.startswith(sentence1) or sentence1.startswith(sentence2) or
                    sentence2.endswith(sentence1) or sentence1.endswith(sentence2)):

                    # print("\nPossible match:")
                    # print("→ Sentence 1:", sentence1)
                    # print("→ Indexes 1:", indexes1)
                    # print("→ Sentence 2:", sentence2)
                    # print("→ Similarity:", round(similarity, 3))

                    try:
                        overlap = sentence2
                        overlap_index = get_charwise_token_ids(sentence1, sentence2, indexes1)
                    except ValueError as e:
                        print(f"Skipping due to index mismatch: {e}")
                        continue

                    results.add((file_name, overlap, gold_label, overlap_index))
                    matched_not_any.add((file_name, sentence1, indexes1))
                    matched_dev.add(sentence2)
                    break

    remaining_not_matching_any = [pair for pair in not_matching_any if pair not in matched_not_any]
    remaining_dev = [s for s in dev_remaining_unmatched if s[0] not in matched_dev]

    return results, remaining_not_matching_any, remaining_dev

In [20]:
results_match_dev, remaining_any2, final_remaining_dev = find_and_remove_overlapping_matches(not_matching_any, dev_remaining_unmatched, threshold=0.8)

Skipping due to index mismatch: Sentence2 not found in sentence1:
→ sentence1: SocietaSolidarietàVivaVialeBruzzi117MilanoAll'attenzionedellaSpettabileSocietaSolidarietàVivaMichiamoMicheleRossiesonolaureatoinlingueconspezializzazioneinlinguaitalianaefrancesepressol'UniversitaUniversitàXaCittàY.
→ sentence2: leSocietaSolidarietàVivaVialeBruzzi117MilanoAll'attenzionedellaSpettabileSocietaSolidarietàVivaMichiamoMicheleRossiesonolaureatoinlingueconspezializzazioneinlinguaitalianaefrancesepressol'UniversitaUniversitàXaCittàY.
Skipping due to index mismatch: Sentence2 not found in sentence1:
→ sentence1: iosonoMariaRossiinUngheria,vivoaCittàX.
→ sentence2: Gabriella,iosonoMariaRossiinUngheria,vivoaCittàX.
Skipping due to index mismatch: Sentence2 not found in sentence1:
→ sentence1: 0345-11111111Spett."SolidarietàViva"VialeBruzzi11721023MilanoConcerne:richiestauncampdilavoroEgregiSigniori,Sonounastudentessadistoriaall'UniversitàdiCittàX.
→ sentence2: MariaRossiCittàX,27maggio2011StradaMaggior

In [21]:
for item in results_match_dev:
    print(item)
    print()

('1385_0001734.exb', 'Chetipodilavorovuoifare?', 'ccccccc', 'T35T35T35T36T36T36T36T37T37T38T38T38T38T38T38T39T39T39T39T40T40T40T40T41')

('1365_0100262.exb', 'Sperodiricevereprestolavostrarisposta/ilvostrodesiderio!!', 'ccccccccccccc', 'T169T169T169T169T169T170T170T171T171T171T171T171T171T171T171T172T172T172T172T172T172T173T173T174T174T174T174T174T174T175T175T175T175T175T175T175T175T176T177T177T178T178T178T178T178T178T179T179T179T179T179T179T179T179T179T180T181')

('1325_9000089.exb', 'Benchésiadettocheèfacilediraggiungereillagoeramoltolontano.', 'cicciciciccccc', 'T132T132T132T132T132T132T133T133T133T134T134T134T134T134T135T135T135T136T137T137T137T137T137T137T138T138T139T139T139T139T139T139T139T139T139T139T139T141T141T142T142T142T142T143T143T143T144T144T144T144T144T145T145T145T145T145T145T145T146')

('1385_0001766.exb', 'SevuoipossiamofarequalcosasabatoseraperesempioandareacenainunnuovoristorantetailandeseaCittàX?', 'ccicccciccccccccccccc', 'T40T40T41T41T41T41T43T43T43T43T43T43T43T43T

In [22]:
list_match_dev = list(results_match_dev)
matching_dev.extend(list_match_dev)
print(len(matching_dev))

719


In [23]:
sentences = [t[1] for t in matching_dev]
sentence_counts = Counter(sentences)

duplicates = {sent: count for sent, count in sentence_counts.items() if count > 1}

print("Duplicate sentences and their counts:")
for sent, count in duplicates.items():
    print(f"{repr(sent)} → {count} times")

print()

sentence_indices = defaultdict(list)

for idx, sentence in enumerate(all_combined_sentences_dev):
    sentence_indices[sentence].append(idx)

duplicates = {s: idxs for s, idxs in sentence_indices.items() if len(idxs) > 1}

print("Multiged duplicate sentences and their counts:")
for sentence, indices in duplicates.items():
    print(f"{repr(sentence)} → {len(indices)} times")

Duplicate sentences and their counts:
'Vorreiaiutarti,mahobisognodiqualcheinformazioneinpiù.' → 4 times
'CaroMichele,grazieperlatuaemail.' → 3 times
'Andiamoacenainunristorantetailandese.' → 2 times
'Comestai?' → 4 times
'Grazieperlatuaemail.' → 2 times
"Com'èlatuacittà?" → 2 times
'SonoaCittàXperfareuncorsodiitaliano.' → 2 times
'Chenepensi?' → 3 times
'Aspettolatuarisposta!' → 2 times
'CaroDaniele,comestai?' → 2 times

Multiged duplicate sentences and their counts:
('!', 'c') → 3 times
('Aspettolatuarisposta!', 'ccccc') → 2 times
('Chenepensi?', 'cccc') → 3 times
('Vorreiaiutarti,mahobisognodiqualcheinformazioneinpiù.', 'cccccccccccc') → 4 times
('Comestai?', 'ccc') → 2 times
('CaroDaniele,comestai?', 'cccccc') → 2 times
('Andiamoacenainunristorantetailandese.', 'cccccccc') → 2 times
('CaroMichele,grazieperlatuaemail.', 'ccccccccc') → 3 times
('Comestai?', 'icc') → 2 times
('SonoaCittàXperfareuncorsodiitaliano.', 'ccccccccccc') → 2 times
("Com'èlatuacittà?", 'cccccc') → 2 times


In [24]:
results_match_train, remaining_any1, final_remaining_train = find_and_remove_overlapping_matches(not_matching_any, train_remaining_unmatched, threshold=0.8)

Skipping due to index mismatch: Sentence2 not found in sentence1:
→ sentence1: CaroFrancesco,grazieperlatualettera.
→ sentence2: '12CaroFrancesco,grazieperlatualettera.
Skipping due to index mismatch: Sentence2 not found in sentence1:
→ sentence1: CaroFrancesco,grazieperlatualettera.
→ sentence2: '12CaroFrancesco,grazieperlatualettera.
Skipping due to index mismatch: Sentence2 not found in sentence1:
→ sentence1: 10Julyal28JulyPotrebberoinformarmicheinqualemodosonoabbinatequesticorsi?
→ sentence2: dal10Julyal28JulyPotrebberoinformarmicheinqualemodosonoabbinatequesticorsi?
Skipping due to index mismatch: Sentence2 not found in sentence1:
→ sentence1: CantonTicinoResidencealLagoAllacorteseattenzionedelDirettoreEgregioSignoreDirettore,cisiamosistematinelVostroalbergodalle5alle20agosto.
→ sentence2: MariaRossi12345CittàX,StradaMaggioren.UngheriaCittàX,13.08.2011CantonTicinoResidencealLagoAllacorteseattenzionedelDirettoreEgregioSignoreDirettore,cisiamosistematinelVostroalbergodalle5alle20ag

In [25]:
print(len(results_match_train))
print(len(results_match_dev))

print(len(final_remaining_train))
print(len(final_remaining_dev))

147
19
351
39


("MariaRossiStradaMaggiore12345CittàXSolidarietàVivaVialeBruzzi11721023MilanoOggetto:Offrirel'aiutoGentilissimo/aSignor/a,stoscrivendoquestaletteradopochehotrovatoVostroannuncionellCorrieredellaSera,al28m.GentileSignore!",
 'cccccccccccccccccicicciccccccccccciciccccicccccc')

In [26]:
sents1 = {s for _, s, _ in remaining_any1}
sents2 = {s for _, s, _ in remaining_any2}

common_sentences = sents1 & sents2

combined_common_tuples = list({
    tup for lst in (remaining_any1, remaining_any2)
    for tup in lst
    if tup[1] in common_sentences
})

print(len(combined_common_tuples))
print(combined_common_tuples[0])

9665
('1325_1001010.exb', "Laprimacosaè,l'appartamentoèvecchio,dopol'ariacondizionatanonhafunzionataesopprattuttolapiscinaerasempresovraffollata.Illagoeralontanodall'appartamentoederanopochicollegamenti.", 'T87T87T88T88T88T88T88T89T89T89T89T90T91T92T92T93T93T93T93T93T93T93T93T93T93T93T93T94T95T95T95T95T95T95T95T96T97T97T97T97T98T98T99T99T99T99T100T100T100T100T100T100T100T100T100T100T100T100T101T101T101T102T102T103T103T103T103T103T103T103T103T103T103T104T105T105T105T105T105T105T105T105T105T105T105T105T106T106T107T107T107T107T107T107T107T108T108T108T109T109T109T109T109T109T110T110T110T110T110T110T110T110T110T110T110T110T110T111T112T112T113T113T113T113T114T114T114T115T115T115T115T115T115T115T116T116T116T116T116T117T117T117T117T117T117T117T117T117T117T117T117T118T118T120T120T120T120T120T121T121T121T121T121T122T122T122T122T122T122T122T122T122T122T122T122T123')


In [27]:
possible_matches, common_unm_1, fin_fin_remaining_dev = find_and_remove_overlapping_matches(combined_common_tuples, final_remaining_dev, threshold=0.6)

Skipping due to index mismatch: Sentence2 not found in sentence1:
→ sentence1: .
→ sentence2: S.
Skipping due to index mismatch: Sentence2 not found in sentence1:
→ sentence1: "SolidarietàViva"VialeBruzzi11721023MilanoConcerne:richiestauncampdilavoroEgregiSigniori,Sonounastudentessadistoriaall'UniversitàdiCittàX.
→ sentence2: MariaRossiCittàX,27maggio2011StradaMaggiore36612345CittàXtel.0345-11111111Spett."SolidarietàViva"VialeBruzzi11721023MilanoConcerne:richiestauncampdilavoroEgregiSigniori,Sonounastudentessadistoriaall'UniversitàdiCittàX.
Skipping due to index mismatch: Sentence2 not found in sentence1:
→ sentence1: iosonoMariaRossiinUngheria,vivoaCittàX.
→ sentence2: Gabriella,iosonoMariaRossiinUngheria,vivoaCittàX.
Skipping due to index mismatch: Sentence2 not found in sentence1:
→ sentence1: Aspettoconimpatienza.
→ sentence2: Assolutamente!Aspettoconimpatienza.
Skipping due to index mismatch: Sentence2 not found in sentence1:
→ sentence1: 0345-11111111Spett."SolidarietàViva"VialeB

In [28]:
for item in possible_matches:
    print(item)
    print()

('1365_0100256.exb', 'Conosciunabellapensionevicinodate?', 'cccccicc', 'T49T49T49T49T49T49T49T50T50T50T51T51T51T51T51T52T52T52T52T52T52T52T52T53T53T53T53T53T53T54T54T55T55T56')

('1395_0000354.exb', 'Sonostatofacendouncorsodiitaliano.', 'iicccccc', 'T23T23T23T23T24T24T24T24T24T25T25T25T25T25T25T25T26T26T27T27T27T27T27T28T28T29T29T29T29T29T29T29T29T30')

('1365_0100167.exb', "perchènonparliunpo'dimatrimonioquandotelefoniconme?", 'iccccicciiic', 'T35T35T35T35T35T35T36T36T36T37T37T37T37T37T38T38T39T39T39T40T40T41T41T41T41T41T41T41T41T41T41T42T42T42T42T42T42T44T44T44T44T44T44T44T44T45T45T45T46T46T47')

('1395_0000587.exb', "quandononc'elezione,faccioungironelcentrodiCittàX.", 'icciccccccccccc', 'T40T40T40T40T40T40T41T41T41T42T42T43T44T44T44T44T44T44T44T45T46T46T46T46T46T46T47T47T48T48T48T48T49T49T49T50T50T50T50T50T50T51T51T52T52T52T52T52T53T54')

('1395_0001028.exb', "Vorreianchesapere:Com'èlatuacittà?", 'cccciccccc', 'T99T99T99T99T99T99T100T100T100T100T100T101T101T101T101T101T101T102T103T

In [29]:
matching_dev.extend(possible_matches)
print(len(matching_dev))

730


In [67]:
print(len(common_unm_1))

9653


In [63]:
print(fin_fin_remaining_dev)

[("MariaRossiStradaMaggiore12345CittàXSolidarietàVivaVialeBruzzi11721023MilanoOggetto:Offrirel'aiutoGentilissimo/aSignor/a,stoscrivendoquestaletteradopochehotrovatoVostroannuncionellCorrieredellaSera,al28m.GentileSignore!", 'cccccccccccccccccicicciccccccccccciciccccicccccc'), ('Lamiaamicapuòvenireconme?', 'cccccccc'), ('nel1987.', 'ccc'), ("Secondomeèbuonostudiarelelingueperchèpossoconosceremoltagenteeavròirapportiall'estero.", 'ccciccciccccciiccci'), ('Quantigiornibisogneràalavora.', 'ccciic'), ('Gennaio2012Ogg.', 'iccc'), ('S.', 'cc'), ('Maggio2010ValtourviaggievacanzeViaF.', 'ccccccccc'), ('!', 'c'), ('!', 'c'), ('!', 'c'), ('MariaRossiCittàX,27maggio2011StradaMaggiore36612345CittàXtel.0345-11111111Spett."SolidarietàViva"VialeBruzzi11721023MilanoConcerne:richiestauncampdilavoroEgregiSigniori,Sonounastudentessadistoriaall\'UniversitàdiCittàX.', 'ccccccccccccccccccccccccccccciicccicicccccccccc'), ("Iotrovobenissimocheildenaroguadagnatosiausatoperfinanziareprogettidisviluppoinpaesicome

In [84]:
def search_in_list_b(char_to_search, sentence_list):
    matches = []
    for file_id, sentence, _ in sentence_list:
        if char_to_search in sentence:
            matches.append((file_id, sentence))
    return matches

char = "x@internet."
matches_with_char = search_in_list_b(char, common_unm_1)

print(f"Sentences in list B containing '{char}':")
for file_id, sentence in matches_with_char:
    print(f"{file_id}: {sentence}\n")



Sentences in list B containing 'x@internet.':
1365_0100469.exb: X1(hotel.x@internet.it)TantisalutiMichele

1365_0100469.exb: Chenepensi?X1(hotel.x@internet.it)

1365_0100469.exb: X1(hotel.x@internet.it)



In [30]:
sentences = [t[1] for t in matching_dev]
sentence_counts = Counter(sentences)

duplicates = {sent: count for sent, count in sentence_counts.items() if count > 1}

print("Duplicate sentences and their counts:")
for sent, count in duplicates.items():
    print(f"{repr(sent)} → {count} times")

print()

sentence_indices = defaultdict(list)

for idx, sentence in enumerate(all_combined_sentences_dev):
    sentence_indices[sentence].append(idx)

duplicates = {s: idxs for s, idxs in sentence_indices.items() if len(idxs) > 1}

print("Multiged duplicate sentences and their counts:")
for sentence, indices in duplicates.items():
    print(f"{repr(sentence)} → {len(indices)} times")

Duplicate sentences and their counts:
'Vorreiaiutarti,mahobisognodiqualcheinformazioneinpiù.' → 4 times
'CaroMichele,grazieperlatuaemail.' → 3 times
'Andiamoacenainunristorantetailandese.' → 2 times
'Comestai?' → 4 times
'Grazieperlatuaemail.' → 2 times
"Com'èlatuacittà?" → 2 times
'SonoaCittàXperfareuncorsodiitaliano.' → 2 times
'Chenepensi?' → 3 times
'Aspettolatuarisposta!' → 2 times
'CaroDaniele,comestai?' → 2 times

Multiged duplicate sentences and their counts:
('!', 'c') → 3 times
('Aspettolatuarisposta!', 'ccccc') → 2 times
('Chenepensi?', 'cccc') → 3 times
('Vorreiaiutarti,mahobisognodiqualcheinformazioneinpiù.', 'cccccccccccc') → 4 times
('Comestai?', 'ccc') → 2 times
('CaroDaniele,comestai?', 'cccccc') → 2 times
('Andiamoacenainunristorantetailandese.', 'cccccccc') → 2 times
('CaroMichele,grazieperlatuaemail.', 'ccccccccc') → 3 times
('Comestai?', 'icc') → 2 times
('SonoaCittàXperfareuncorsodiitaliano.', 'ccccccccccc') → 2 times
("Com'èlatuacittà?", 'cccccc') → 2 times


In [31]:
def process_data(data_list):
    output = []
    
    for data in data_list:
        filename, sentence, gold_labels, indices = data
        
        numeric_indices = []
        i = 0
        while i < len(indices):
            if indices[i] == 'T':
                i += 1
                num_str = ''
                while i < len(indices) and indices[i].isdigit():
                    num_str += indices[i]
                    i += 1
                if num_str:
                    numeric_indices.append(num_str)
            else:
                i += 1
        
        grouped_indices = []
        if not numeric_indices:
            continue
        
        current_group = [numeric_indices[0]]
        for num in numeric_indices[1:]:
            if num == current_group[-1]:
                current_group.append(num)
            else:
                grouped_indices.append(current_group)
                current_group = [num]
        grouped_indices.append(current_group)
        
        start = 0
        word_index_pairs = []
        
        for idx, group in enumerate(grouped_indices):
            word_length = len(group)
            if start + word_length > len(sentence):
                break 
            word = sentence[start:start + word_length]
            unique_index = f"T{group[0]}"
            label = gold_labels[idx]
            word_index_pairs.append([unique_index, word, label])
            start += word_length
        
        output.append({filename: word_index_pairs})
    
    return output

In [32]:
aligned_data = process_data(matching_dev)
print(aligned_data)

[{'1385_0000013.exb': [['T0', 'Puoi', 'c'], ['T1', 'dirmi', 'c'], ['T2', 'dove', 'c'], ['T3', 'lavori', 'c'], ['T4', 'adesso', 'c'], ['T5', 'e', 'c'], ['T6', 'che', 'c'], ['T7', 'tipo', 'c'], ['T8', 'di', 'c'], ['T9', 'lavoro', 'c'], ['T10', 'ti', 'c'], ['T11', 'piaccerebbe', 'i'], ['T12', '?', 'c']]}, {'1395_0001067.exb': [['T24', 'Tutti', 'c'], ['T25', 'della', 'i'], ['T26', 'mia', 'c'], ['T27', 'famiglia', 'c'], ['T28', 'stanno', 'c'], ['T29', 'benissimo', 'c'], ['T30', '.', 'c']]}, {'1385_0000123.exb': [['T9', 'Vorrei', 'c'], ['T10', 'aiutarti', 'c'], ['T11', ',', 'c'], ['T12', 'ma', 'c'], ['T13', 'ho', 'c'], ['T14', 'bisogno', 'c'], ['T15', 'di', 'c'], ['T16', 'qualche', 'c'], ['T17', 'informazione', 'c'], ['T18', 'in', 'c'], ['T19', 'più', 'c'], ['T20', '.', 'c']]}, {'1385_0001161.exb': [['T9', 'Vorrei', 'c'], ['T10', 'aiutarti', 'c'], ['T11', ',', 'c'], ['T12', 'ma', 'c'], ['T13', 'ho', 'c'], ['T14', 'bisogno', 'c'], ['T15', 'di', 'c'], ['T16', 'qualche', 'c'], ['T17', 'informaz

In [33]:
def merge_aligned_and_th1th2_with_gaps_only(aligned_data, th1_th2):
    th1_th2_dict = {
        filename: {t[0]: t[1:] for t in data}
        for item in th1_th2
        for filename, data in item.items()
    }

    merged_all = []
    insertion_corrections = []

    for item in aligned_data:
        filename, token_list = list(item.items())[0]

        if filename not in th1_th2_dict:
            print(f"Warning: {filename} not found in th1_th2.")
            continue

        th_lookup = th1_th2_dict[filename]

        try:
            aligned_tids = [int(entry[0][1:]) for entry in token_list]
        except Exception as e:
            print(f"Error parsing TIDs in {filename}: {e}")
            continue

        aligned_tid_set = {f"T{num}" for num in aligned_tids}
        full_range = {f"T{i}" for i in range(min(aligned_tids), max(aligned_tids) + 1)}
        missing_tids = sorted(full_range - aligned_tid_set, key=lambda x: int(x[1:]))

        for tid in missing_tids:
            if tid in th_lookup:
                insertion_corrections.append((filename, tid, th_lookup[tid]))

        merged = []
        for tid, token, label in token_list:
            th_info = th_lookup.get(tid, ('', '', '', '', '', ''))
            merged.append((tid, token) + th_info + (label,))

        merged_all.append({filename: merged})

    return merged_all, insertion_corrections


In [34]:
halleluja,insertion_corrections = merge_aligned_and_th1th2_with_gaps_only(aligned_data, th1_th2)

In [65]:
print(halleluja[100])

{'1365_0100458.exb': [('T116', 'Inoltre', 'Inoltre', '', '', '', 'O_Punct', 'c'), ('T118', 'devo', 'devo', '', '', '', '', 'i'), ('T119', 'augurarti', 'augurarti', '', '', '', '', 'c'), ('T120', 'primo', 'prima', 'CHA', '', '', 'G_Pos', 'i'), ('T121', 'che', 'che', '', '', '', '', 'c'), ('T123', 'ne', 'ne', '', '', '', '', 'i'), ('T124', 'dimentico', 'dimentico', '', '', '', 'G_Refl', 'c'), ('T125', '.', '.', '', '', '', '', 'c')]}


In [67]:
# def export_to_tsv(merged_data, output_path):
#     rows = []
#     for entry in merged_data:
#         filename, token_data = list(entry.items())[0]
#         for row in token_data:
#             row_list = list(row)
#             row_trimmed = row_list[1:8]
#             rows.append([filename] + row_trimmed)
#         rows.append([''] * 8)

#     columns = ['filename', 'learner', 'TH1', 'TH1Diff', 'TH2', 'TH2Diff', 'markable_scheme', 'gold_label']

#     df = pd.DataFrame(rows, columns=columns)
#     df.to_csv(output_path, sep='\t', index=False)

# export_to_tsv(halleluja, 'processed_merlin.tsv')